In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [2]:
IMAGE_SIZE = [224, 224]
path="../pokemon_dataset/"

In [3]:
from keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             vertical_flip=True,
                             rotation_range=30)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(path + 'train', 
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

test_it = datagen.flow_from_directory(path + 'test',  
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

Found 126 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [4]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras import Sequential

In [5]:
model = Sequential()
model.add(Conv2D(32, (5,5),padding='valid',activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [6]:
from keras.losses import CategoricalCrossentropy
model.compile(
  optimizer='adam',
  loss=CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.Precision()])

In [7]:
history = model.fit( 
  train_it,
  validation_data=test_it,
  epochs=5)

Epoch 1/5
 7/63 [==>...........................] - ETA: 13s - loss: 31.6283 - precision: 0.5000

KeyboardInterrupt: 